**Install**

In [2]:
!pip install html5lib
!pip install lxml

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 2.8 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.lau

**Import**

In [3]:
import sys
from selenium import webdriver
import time

import bs4
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json


**Setting**

In [28]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [29]:
SESSION = requests.Session()
HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/48.0.2564.109 Safari/537.36')
}

BASE_URL = 'https://finance.vietstock.vn/BID/tai-chinh.htm?tab=BCTT'


In [45]:
print('- webdriver is starting ... ')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
browser.get(BASE_URL)
time.sleep(10)  


- webdriver is starting ... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [49]:
# Thao tác lùi trang từ năm 2020 về năm 2006
showdata_option = browser.find_element_by_xpath("/html/body/div[2]/div[11]/div/div[4]/div/div[2]/div/div/div[3]/div/div[1]/select")
showdata_option.click()
showdata_year_option = browser.find_element_by_xpath("/html/body/div[2]/div[11]/div/div[4]/div/div[2]/div/div/div[3]/div/div[1]/select/option[1]")
showdata_year_option.click()
time.sleep(5)
# Click 4 lần
for i in range(1,5):
    btn_back_data =  browser.find_element_by_xpath("/html/body/div[2]/div[11]/div/div[4]/div/div[2]/div/div/div[3]/div/div[3]/div[2]")
    btn_back_data.click()
    time.sleep(10)  

# Bắt đầu lấy dữ liệu từ năm 2006 -> 2020
data = []
for i in range(1,5):
    btn_continue_data =  browser.find_element_by_xpath("/html/body/div[2]/div[11]/div/div[4]/div/div[2]/div/div/div[3]/div/div[3]/div[3]")
    btn_continue_data.click()
    time.sleep(10)  
    page = browser.page_source
    soup = BeautifulSoup(page, 'html.parser')

    # Hiển thị tên bảng và các năm
    name_table = []
    main_title_classes = soup.findAll('h4', class_='no-wrapper')
    for main_title_class in main_title_classes:
      name_table.append(main_title_class.text)

    # Hiển thị các năm
    sub_title_classes = soup.findAll(class_='text-center col-90')
    years = []
    for sub_title_class in sub_title_classes:
      title_year =  sub_title_class.find("b")
      years.append(title_year.text)
      print(title_year.text)

    # Nội dung các mục cần hiên thị
    data.append({"definition_name": name_table[0], "properties_1": years[0], "properties_2": years[1] , "properties_3": years[2], "properties_4": years[3]})   
    content_classes  = soup.findAll(class_ ="g-c-r")
    for content_class in content_classes:
      sub_title =  content_class.find("span")
      if(sub_title.text == 'Thu nhập trên mỗi cổ phần của 4 quý gần nhất (EPS)'):
        break
      else:
        if(sub_title.text == 'Lãi cơ bản trên cổ phiếu (VNÐ)'):  
          print(sub_title.text)
          name_mean = sub_title.text
          sub_num = content_class.find_all(class_ = "text-right")
          properties = []
          for num in sub_num:
            if(num.text == '-'):
              properties.append(0)
              print(0)
            else:
              properties.append(num.text)
              print(num.text)
          data.append({"definition_name": name_mean, "properties_1": properties[0], "properties_2": properties[1] , "properties_3": properties[2],
          "properties_4": properties[3]})
          data.append({"definition_name": name_table[1], "properties_1": years[0], "properties_2": years[1] , "properties_3": years[2], "properties_4": years[3]}) 
        else:
          if(sub_title.text == 'Lãi cơ bản trên cổ phiếu (VNÐ)'):  
            continue
          else:
            print(sub_title.text)
            name_mean = sub_title.text
            sub_num = content_class.find_all(class_ = "text-right")
            properties = []
            for num in sub_num:
              if(num.text == '-'):
                properties.append(0)
                print(0)
              else:
                properties.append(num.text)
                print(num.text)
          data.append({"definition_name": name_mean, "properties_1": properties[0], "properties_2": properties[1] , "properties_3": properties[2],
          "properties_4": properties[3]})      
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')

Năm 2005
Năm 2006
Năm 2007
Năm 2008
Năm 2005
Năm 2006
Năm 2007
Năm 2008
Năm 2005
Năm 2006
Năm 2007
Năm 2008
Thu nhập lãi thuần
2,929,368
2,432,434
4,856,449
6,243,550
Lãi/lỗ thuần từ hoạt động dịch vụ
246,598
414,380
624,190
1,002,888
Lãi/lỗ thuần từ hoạt động kinh doanh ngoại hối và vàng
44,224
104,777
139,647
790,779
Lãi/lỗ thuần từ mua bán chứng khoán kinh doanh
499
221,513
159,830
-838,813
Lãi/lỗ thuần từ mua bán chứng khoán đầu tư
599,579
993,846
0
0
Lãi/lỗ thuần từ hoạt động khác
-380,383
-23,331
1,193,932
-99,722
Thu nhập từ góp vốn, mua cổ phần
24,526
33,195
16,629
-7,588
Chi phí hoạt động
691,845
977,407
1,564,594
2,186,974
Lợi nhuận thuần từ hoạt động kinh doanh trước chi phí dự phòng rủi ro tín dụng
2,772,566
3,199,407
5,426,083
4,904,120
Chi phí dự phòng rủi ro tín dụng
2,031,687
1,993,491
3,397,837
2,553,515
Tổng lợi nhuận trước thuế
740,879
1,205,916
2,028,246
2,350,605
Lợi nhuận sau thuế
559,993
1,075,878
1,531,416
1,979,392
Lợi nhuận sau thuế của cổ đông của Ngân hàng m

In [50]:
data

[{'definition_name': 'Kết quả kinh doanh',
  'properties_1': 'Năm 2005',
  'properties_2': 'Năm 2006',
  'properties_3': 'Năm 2007',
  'properties_4': 'Năm 2008'},
 {'definition_name': 'Thu nhập lãi thuần',
  'properties_1': '2,929,368',
  'properties_2': '2,432,434',
  'properties_3': '4,856,449',
  'properties_4': '6,243,550'},
 {'definition_name': 'Lãi/lỗ thuần từ hoạt động dịch vụ',
  'properties_1': '246,598',
  'properties_2': '414,380',
  'properties_3': '624,190',
  'properties_4': '1,002,888'},
 {'definition_name': 'Lãi/lỗ thuần từ hoạt động kinh doanh ngoại hối và vàng',
  'properties_1': '44,224',
  'properties_2': '104,777',
  'properties_3': '139,647',
  'properties_4': '790,779'},
 {'definition_name': 'Lãi/lỗ thuần từ mua bán chứng khoán kinh doanh',
  'properties_1': '499',
  'properties_2': '221,513',
  'properties_3': '159,830',
  'properties_4': '-838,813'},
 {'definition_name': 'Lãi/lỗ thuần từ mua bán chứng khoán đầu tư',
  'properties_1': '599,579',
  'properties_2

In [51]:
with open('Craw_Finace.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

url from csv to excel: https://data.page/json/csv